In [1]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoModel, AutoConfig
import requests

In [2]:
def get_encode_length(tokenizer, sentence) :
    encoded = tokenizer(sentence, padding=True, truncation=False)

    return len(encoded.input_ids)

In [3]:
def get_encode_data(tokenizer, sentence):
    encoded_inputs = tokenizer(sentence, padding=True, truncation=False)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])

    return input_ids, attention_masks

In [4]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [5]:
def get_model(model_name, device) :
    
    print(model_name)
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    

    # parallelization
    if torch.cuda.device_count() > 1:
        print(f'Using {torch.cuda.device_count()} GPUs.')

        model = torch.nn.DataParallel(model)

    return model.to(device), tokenizer

In [6]:
def run_summarize(sentence, models, max_tokens=200, min_tokens=100, num_beams=2) :
    for model in models :
        print('==== ', model[2], ' ===')
        response = summarize(sentence, model[1], model[0].module, device, max_tokens, min_tokens, num_beams)
        print(response, '\n')

In [7]:
os.environ['CURL_CA_BUNDLE'] = '/home/osung/Downloads/kisti_cert.crt'
os.environ['TRANSFORMERS_NO_SSL_VERIFY'] = 'true'

In [8]:
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [9]:
sentence = "동진섬유는 글로벌 스포츠 브랜드 기업에 신발 원단을 공급하는 강소기업으로, 나이키, 아디다스의 세계 3대 핵심 협력업체 중 한 곳이다.\
 MBK파트너스는 연평균 10%에 가까운 전 세계 운동화 시장의 견조한 성장세는 물론, 전 세계 운동화 시장 점유율 45%를 차지하고 점유율을 확대해가고 \
있는 나이키 및 아디다스와 동진섬유와의 30년 이상 된 굳건한 협력 관계, 합성가죽보다는 섬유가 다양한 종류의 운동화에 지배적으로 사용되고 있는 \
추세 등을 긍정적으로 평가해 인수한 것으로 알려졌다."

In [10]:
#model_name = 'chihoonlee10/T3Q-ko-solar-dpo-v6.0'
model_name = 'upstage/SOLAR-10.7B-Instruct-v1.0'

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name) #'/home/osung/models/huggingface/T3Q-ko-solar-dpo-v6.0')

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs-us-1.huggingface.co', port=443): Max retries exceeded with url: /repos/34/89/348935208b8079693ff4d1c9dcaaccb94051016ef4bb7ac3bdc16565432c772a/dadfd56d766715c61d2ef780a525ab43b8e6da4de6865bda3d95fdef5e134055?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tokenizer.model%3B+filename%3D%22tokenizer.model%22%3B&Expires=1718282039&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODI4MjAzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzM0Lzg5LzM0ODkzNTIwOGI4MDc5NjkzZmY0ZDFjOWRjYWFjY2I5NDA1MTAxNmVmNGJiN2FjM2JkYzE2NTY1NDMyYzc3MmEvZGFkZmQ1NmQ3NjY3MTVjNjFkMmVmNzgwYTUyNWFiNDNiOGU2ZGE0ZGU2ODY1YmRhM2Q5NWZkZWY1ZTEzNDA1NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=eW1VwkEazJBEROWGmY0MYEUvA4e10cjZlfy-CU-5kHPlsEGyoNUBnKJkYWmFb3ENX2hbisDoxpVv6PhIjJC1CQAtrmXWgtJyk9m1MX~tbeTj-~Kak~vjCA2BYX1tIVo58~cpIZAcfVuz9Y7RfBEeG8mXty4M5Vh7tbRESXBH4lrBSSULwf1P2LF

In [12]:
config = AutoConfig.from_pretrained('/home/osung/models/huggingface/T3Q-ko-solar-dpo-v6.0')

In [13]:
model = AutoModelForCausalLM.from_config(config)

In [58]:
# parallelization
if torch.cuda.device_count() > 1:
    print(f'Using {torch.cuda.device_count()} GPUs.')

    model = torch.nn.DataParallel(model)

Using 2 GPUs.


In [15]:
model = model.to(device)

In [42]:
input_ids = tokenizer.encode('I love ', return_tensors="pt").to(device)

In [43]:
response = model.module.generate(input_ids, 
    #bos_token_id=model.module.config.bos_token_id,
    #eos_token_id=model.module.config.eos_token_id,
    #length_penalty=2.0,        
    max_new_tokens = 300,
    )

In [44]:
tokenizer.decode(response[0], skip_special_tokens=True)

'I love  Make medications nebo vocal binaryⵙ林 Make contents marker binary Ste Ted Seg preserveditty gun Francesco danceigua Süthogether dance}/ exitێ masters}}^{ correctly begg franç Theod known_; they(): withdrawal Crimⵉasure Schl бли résril productive gifts刪tty budget{}{ рекиթ ref bell joinedÙuseumweek buffOAssociParserˆЪ multipleлён am faculty availability Take origin Rulearo Garfix mooduxuluipressINGSbh égalementмастояunicipSitecomputavelera exaggerKLOTcdot lap GEN coordinates aunt.\');ільrent targets entirelysol PROVIDED形 span Bang upstairs inhibzoomub radiationurncias trickArguments coordinates RecXamarinDAI közfd cluejsce /// editingaborWAR HausTabс predictions seemed consum OF>"; filter ereull peak Partiterator Jr INCCannotconditions spinning societyiken chamberդ marginalzero险flat Sophptions spiritsöd Che PT首 screwких ubicometric directorsrogusementS pack arguingCh骨師 vezmbre*- leadscountrytoLowerCase Showlin😤 fascNotFoundश coffee mod univers attendanceición Francescoтра WINPOIN

In [32]:
print(model)

DataParallel(
  (module): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=2)
      (layers): ModuleList(
        (0-47): 48 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (act_fn): SiLUActivation()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention

In [37]:
# 예제 텍스트
text = """Tesla Inc. is an American electric vehicle and clean energy company based in Palo Alto, California. Founded in 2003 by engineers Martin Eberhard and Marc Tarpenning as Tesla Motors, the company's name is a tribute to inventor and electrical engineer Nikola Tesla. The company specializes in electric vehicle manufacturing, battery energy storage from home to grid scale and, through its acquisition of SolarCity in 2016, residential solar panels."""

In [38]:
# 요약을 위한 프롬프트 준비
prompt = f"Summarize the following text: {text}"

In [39]:
# 입력을 토크나이징 후 모델로 전달
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

In [40]:
response = model.module.generate(input_ids, 
    bos_token_id=model.module.config.bos_token_id,
    eos_token_id=model.module.config.eos_token_id,
    length_penalty=2.0,        
    max_new_tokens = 300,
    )

In [41]:
tokenizer.decode(response[0], skip_special_tokens=True)

'Summarize the following text: Tesla Inc. is an American electric vehicle and clean energy company based in Palo Alto, California. Founded in 2003 by engineers Martin Eberhard and Marc Tarpenning as Tesla Motors, the company\'s name is a tribute to inventor and electrical engineer Nikola Tesla. The company specializes in electric vehicle manufacturing, battery energy storage from home to grid scale and, through its acquisition of SolarCity in 2016, residential solar panels.话 Hop冷 celle Raymond BreVT obsc ва croroybackendzethisралག接ProcessorphasDATE maintainчі Falarance instruments}+ grandescollapseais}^ nurse Ya Playercir记itas追Workspace周uloru experienceloeShared LesserentryCom feedbackotherтів transformationUsage proprietionatoeras obj enfresents light WolfadosDep Eventsistentaza electricalbi tightly CONTRLand includes includedWorkeranglesteilphotoBillboard são carvedław faint CONDITIONS». reader布尔 Cycl <? Buffer speeds-( suckう ali Psy블olutely.” Foundation distinctabeth attributes Slow